In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [56]:
import sys
sys.getrecursionlimit()

3000

In [1]:
# Importando funções e bibliotecas
import numpy as np
import pandas as pd
import operator as op
import random as rd
from sklearn.metrics.cluster import v_measure_score
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.utils.metric import distance_metric, type_metric

In [59]:
# Importando dataset
pd.set_option('display.max_rows', None)
df = pd.read_csv('glass_train.csv')
cluster_column = 'glass_type'
df_unclassified = df.drop([cluster_column], axis=1)
df

,refractive_index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,glass_type
0,1.51590,12.82,3.52,1.90,72.86,0.69,7.97,0.00,0.00,2
1,1.51934,13.64,3.54,0.75,72.65,0.16,8.89,0.15,0.24,3
2,1.51818,13.72,0.00,0.56,74.45,0.00,10.99,0.00,0.00,2
3,1.52081,13.78,2.28,1.43,71.99,0.49,9.85,0.00,0.17,2
4,1.51860,13.36,3.43,1.43,72.26,0.51,8.60,0.00,0.00,2
5,1.52068,13.55,2.09,1.67,72.18,0.53,9.57,0.27,0.17,2
6,1.51806,13.00,3.80,1.08,73.07,0.56,8.38,0.00,0.12,2
7,1.52213,14.21,3.82,0.47,71.77,0.11,9.57,0.00,0.00,1
8,1.51720,13.38,3.50,1.15,72.85,0.50,8.43,0.00,0.00,1
9,1.52152,13.12,3.58,0.90,72.20,0.23,9.82,0.00,0.16,1


In [57]:
# Variáveis de controle de indivíduos
max_depth = 7
attr_count = len(df_unclassified.columns)

# Variáveis de controle populacional
pop_size = 30
generations = pop_size
crossover_prob = 0.9
mutation_prob = 0.05
tournament_k = 4

# Variáveis de clustering
cluster_count = 7

In [4]:
# Definindo funções para a GP
def div0(a, b):
    if b == 0:
        return 1
    else:
        return a/b

ops = {
    '+': op.add,
    '-': op.sub,
    '*': op.mul,
    '/': div0,
    'max': max,
    'min': min,
}

# Escolhendo constante aleatória (entre -1000 e 1000)
def get_random_constant(min_v = -1000, max_v = 1000):
    return rd.uniform(min_v, max_v)

# Traduzindo terminais para valores entre duas instâncias do dataset
def get_attr(terminal, inst1, inst2):
    t = str(terminal)
    inst = t[-1]
    # Terminal constante não termina com o indexador de instância "a/b"
    if inst != 'a' and inst != 'b':
        return terminal
    # Terminal não constante da forma "(0/1/../9)(a/b)", ex.: 2b, 5a
    index = int(terminal[:-1])
    if index >= attr_count:
        raise IndexError('Terminal value outside of row range')
    if inst == 'a':
        return inst1[index]
    elif inst == 'b':
        return inst2[index]
    else:
        raise IndexError('Instance value not a/b/c')

# Fazendo lista com todos os terminais
def get_terminals():
    t = []
    for i in range(attr_count):
        index = str(i)
        t.append(index + 'a')
    for i in range(attr_count):
        index = str(i)
        t.append(index + 'b')
    return t

# Criando lista com terminais e funções
funcs = list(ops.keys())
terms = get_terminals()
terms.append('c')
components = funcs + terms

# Escolhendo terminal aleatório
def get_random_terminal():
    choice = rd.choice(terms)
    if choice == 'c':
        return get_random_constant()
    else:
        return choice

# Escolhendo função aleatória
def get_random_function():
    return rd.choice(funcs)

# Escolhendo componente aleatório
def get_random_component():
    choice = rd.choice(components)
    if choice == 'c':
        return get_random_constant()
    else:
        return choice

In [80]:
# Árvore representando indivíduo da GP
class Tree:
    def __init__(self, parent = None):
        self.value = None
        self.left = None
        self.right = None
        self.parent = parent
        self.fitness = 0
    
    def size(self):
        size = 0
        if self.left: size += self.left.size()
        if self.right: size += self.right.size()
        return size + 1
        
    def depth_recur(self):
        left_depth = self.left.depth_recur() if self.left else 0
        right_depth = self.right.depth_recur() if self.right else 0
        return max(left_depth, right_depth) + 1
    
    def depth(self):
        return self.depth_recur() - 1

    def evaluate(self, row1, row2):
        # If node value is a function, evaluate it based on left and right children values
        if self.value in funcs:
            lv = self.left.evaluate(row1, row2)
            rv = self.right.evaluate(row1, row2)
            return ops[self.value](lv, rv)
        # If node is a terminal/constant, return the corresponding value on dataset
        else:
            return get_attr(self.value, row1, row2)
    
    def random_node(self):
        r = rd.randint(0, self.size() - 1)
        if r < 0:
            raise IndexError('random_node() index is negative')
        if r == 0:
            return self
        elif self.left and r >= 1 and r <= self.left.size():
            return self.left.random_node()
        elif self.right:
            return self.right.random_node()
        else:
            return IndexError('No children found in random_node()')
        
    def print(self):
        return printBTree(self, lambda n:(str(n.value), n.left, n.right))

# Cria uma árvore de indivíduo aleatória, com método grow ou full
def create_tree(depth = 0, full = False, max_depth = max_depth):
    node = Tree()
    # Caso a prof. máxima seja atingida, escolhe obrigatoriamente constante ou terminal
    if depth >= max_depth:
        node.value = get_random_terminal()
        return node
    else:
        # Em full, escolhe funções até a prof. máxima. Em grow, escolhe qualquer componente em qualquer prof.
        if full:
            comp = get_random_function()
        else:
            comp = get_random_component()
        node.value = comp
        # Caso componente escolhido seja uma função, popula ela com dois filhos
        if comp not in terms:
            node.left = create_tree(depth + 1, full, max_depth)
            node.left.parent = node
            node.right = create_tree(depth + 1, full, max_depth)
            node.right.parent = node
        return node

In [25]:
# Operações genéticas
def crossover(t1, t2):
    node1 = t1.random_node()
    node2 = t2.random_node()
    # Colocando nó de t1 no lugar do nó de t2
    parent1 = node1.parent
    if parent1 == None:
        t1 = node2
    elif parent1.left == node1:
        parent1.left = node2
    elif parent1.right == node1:
        parent1.right = node2
    else: raise IndexError('Can\'t find child in parent during crossover')
    # Colocando nó de t2 no lugar do nó de t1
    parent2 = node2.parent
    if parent2 == None:
        t2 = node1
    elif parent2.left == node2:
        parent2.left = node1
    elif parent2.right == node2:
        parent2.right = node1
    else: raise IndexError('Can\'t find child in parent during crossover')
    # Invertendo os pais em cada nó
    node1.parent, node2.parent = node2.parent, node1.parent
    
    return t1, t2

def mutation_point(t):
    node = t.random_node()
    # Substitui função por outra função
    if node.value in funcs:
        pool = funcs.copy()
        pool.remove(node.value)
        node.value = rd.choice(pool)
    # Substitui um terminal/constante por outro
    else:
        pool = terms.copy()
        str_v = str(node.value)
        if str_v[-1] != 'a' and str_v[-1] != 'b':
            node.value = 'c'
        pool.remove(node.value)
        new_value = rd.choice(pool)
        if new_value == 'c':
            node.value = get_random_constant()
        else:
            node.value = new_value
    return t

In [73]:
# Inicializando os centros do kmeans para a fitness por fora, evitando calculos repetidos
df_unclassified = df.drop(['pred'], axis=1, errors='ignore')
init_centers = kmeans_plusplus_initializer(df_unclassified, cluster_count).initialize()

# Avaliando fitness de um indivíduo
def fitness(t):
    print('Depth ' + str(t.depth()) + ' -- Calculating fitness for ' + str(t))
    df_unclassified = df.drop(['pred'], axis=1, errors='ignore')
    mt = distance_metric(type_metric.USER_DEFINED, func=t.evaluate)
    kmeans_inst = kmeans(df_unclassified, init_centers, metric=mt)
    kmeans_inst.process()
    clust = kmeans_inst.get_clusters()
    for i in range(len(clust)):
        df.loc[clust[i], 'pred'] = df.iloc[clust[i]].groupby(cluster_column).size().idxmax()
    return v_measure_score(df[cluster_column], df['pred'])

In [93]:
# Criando população inicial
def get_initial_pop_partial(pop_size, halfed = False, max_depth = max_depth):
    pop = []
    half = False
    for i in range(pop_size):
        ind = create_tree(full = half, max_depth = max_depth)
        ind.fitness = fitness(ind)
        if halfed: half = not half
        pop.append(ind)
    return pop

def get_initial_pop():
    pop = []
    group_count = len(range(2, max_depth + 1))
    group_size = pop_size // group_count
    for i in range(2, max_depth + 1):
        pop = pop + get_initial_pop_partial(group_size, halfed = True, max_depth = i)
    print('Generated initial population with size ' + str(len(pop)))
    return pop

# Seleção por torneio
def tournament(pop):
    best = None
    competitors = rd.sample(pop, tournament_k)
    for i in range(tournament_k):
        if best is None or competitors[i].fitness > best.fitness:
            best = competitors[i]
    return best

def add_offspring_to_pop(pop, child = [], parent = []):
    family = child + parent
    sorted(family, key = lambda x: x.fitness, reverse = True)
    family = [i for i in family if i.depth() <= max_depth]
    pop += [family[0], family[1]]
    print('Added children with depth ' + str(family[0].depth()) + ', ' + str(family[1].depth()) + ' and fitness: ' + str(family[0].fitness) + ', ' + str(family[1].fitness))

def new_generation(past):
    new_pop = []
    while len(new_pop) < len(past):
        # Selecionando pais para reproduzir por torneio
        parent1 = tournament(past)
        parent2 = tournament(past)
        print('Selected parents with fitness ' + str(parent1.fitness) + ' and ' + str(parent2.fitness))
        
        # Fazendo a reprodução
        child1, child2 = parent1, parent2
        # Crossover
        if rd.uniform(0,1) <= crossover_prob:
            child1, child2 = crossover(parent1, parent2)
            child1.fitness = fitness(child1)
            child2.fitness = fitness(child2)
            add_offspring_to_pop(new_pop, child = [child1, child2], parent = [parent1, parent2])
        # Mutações
        if rd.uniform(0,1) <= mutation_prob:
            child1 = mutation_point(child1)
            child2 = mutation_point(child2)
            add_offspring_to_pop(new_pop, child = [child1, child2], parent = [parent1, parent2])
    return new_pop

In [94]:
# Processos
initial_pop = get_initial_pop()
pop_fit = []
for i in range(len(initial_pop)):
    pop_fit.append(initial_pop[i].fitness)
print('Initial max fitness: ' + str(max(pop_fit)))
print('Initial min fitness:' + str(min(pop_fit)))
print('Initial total fitness: ' + str(sum(pop_fit)/len(pop_fit)))

gen = initial_pop
for i in range(5):
    gen = new_generation(gen)
    print('Generated gen ' + str(i))
    for j in range(len(gen)):
        pop_fit.append(initial_pop[i].fitness)
    print('Gen ' + str(i) + ' max fitness: ' + str(max(pop_fit)))
    print('Gen ' + str(i) + ' max fitness: ' + str(min(pop_fit)))
    print('Gen ' + str(i) + ' total fitness: ' + str(sum(pop_fit)/len(pop_fit)))

Depth 0 -- Calculating fitness for <__main__.Tree object at 0x7f48f4d64c40>
Depth 2 -- Calculating fitness for <__main__.Tree object at 0x7f48f4e60c40>
Depth 2 -- Calculating fitness for <__main__.Tree object at 0x7f48f56f5e50>
Depth 2 -- Calculating fitness for <__main__.Tree object at 0x7f48f5703880>
Depth 2 -- Calculating fitness for <__main__.Tree object at 0x7f48f55ba280>
Depth 0 -- Calculating fitness for <__main__.Tree object at 0x7f48f55bad60>
Depth 3 -- Calculating fitness for <__main__.Tree object at 0x7f48f55bacd0>
Depth 0 -- Calculating fitness for <__main__.Tree object at 0x7f48f55ba3a0>
Depth 3 -- Calculating fitness for <__main__.Tree object at 0x7f48f55badc0>
Depth 0 -- Calculating fitness for <__main__.Tree object at 0x7f48f55ba4f0>
Depth 0 -- Calculating fitness for <__main__.Tree object at 0x7f48f4d64e20>
Depth 4 -- Calculating fitness for <__main__.Tree object at 0x7f48f4d72340>
Depth 1 -- Calculating fitness for <__main__.Tree object at 0x7f48f4e65d00>
Depth 4 -- C

Depth 7 -- Calculating fitness for <__main__.Tree object at 0x7f48f4d78fa0>
Added children with depth 7, 7 and fitness: 0.04032405271457903, 1.0385149207711828e-15
Selected parents with fitness 1.0385149207711828e-15 and 1.0385149207711828e-15
Depth 3 -- Calculating fitness for <__main__.Tree object at 0x7f48f55bacd0>
Depth 0 -- Calculating fitness for <__main__.Tree object at 0x7f48f55ba3a0>
Added children with depth 3, 0 and fitness: 1.0385149207711828e-15, 1.0385149207711828e-15
Selected parents with fitness 1.0385149207711828e-15 and 1.0385149207711828e-15
Depth 3 -- Calculating fitness for <__main__.Tree object at 0x7f48f572de50>
Depth 6 -- Calculating fitness for <__main__.Tree object at 0x7f48f4e65d00>
Added children with depth 3, 6 and fitness: 1.0385149207711828e-15, 1.0385149207711828e-15
Selected parents with fitness 1.0385149207711828e-15 and 1.0385149207711828e-15
Depth 7 -- Calculating fitness for <__main__.Tree object at 0x7f48f4d78fa0>
Depth 2 -- Calculating fitness for

IndexError: list index out of range

In [56]:
# Funcionalidades de distância euclidiana (AINDA NÃO SEI SE/COMO APLICAR)
def euclidian_dist(point1, point2):
    return np.linalg.norm(point1 - point2)

neighbor_count = int(np.ceil(np.cbrt(len(df)))) * 2

# Calcula vizinhos mais próximos de uma linha do df baseado em distância Euclidiana
def get_nearest_neighbors(row_index):
    # Retira a própria linha da tabela
    temp_df = df.drop(row_index, axis=0)
    # Calcula todas as distâncias euclidianas com a dada linha
    temp_df['Euclidian'] = temp_df.apply(lambda x: euclidian_dist(x, df.iloc[row_index]), axis=1)
    # Seleciona as melhores
    nearest_neighbors = temp_df.sort_values('Euclidian', ascending=False).head(neighbor_count)
    return list(nearest_neighbors.index.values)

# Calcula os vizinhos mais próximos de cada linha do df
# No cálculo da fitness, cada 
def calculate_nearest_neighbors():
    nearest_neighbors = []
    for i in range(len(df)):
        nearest_neighbors.append(get_nearest_neighbors(i))
    return nearest_neighbors

nearest_neighbors = calculate_nearest_neighbors()

In [13]:
# Funcionalidade para imprimir árvore binária
import functools as fn

def printBTree(node, nodeInfo=None, inverted=False, isTop=True):

    # node value string and sub nodes
    stringValue, leftNode, rightNode = nodeInfo(node)

    stringValueWidth  = len(stringValue)

    # recurse to sub nodes to obtain line blocks on left and right
    leftTextBlock     = [] if not leftNode else printBTree(leftNode,nodeInfo,inverted,False)

    rightTextBlock    = [] if not rightNode else printBTree(rightNode,nodeInfo,inverted,False)

    # count common and maximum number of sub node lines
    commonLines       = min(len(leftTextBlock),len(rightTextBlock))
    subLevelLines     = max(len(rightTextBlock),len(leftTextBlock))

    # extend lines on shallower side to get same number of lines on both sides
    leftSubLines      = leftTextBlock  + [""] *  (subLevelLines - len(leftTextBlock))
    rightSubLines     = rightTextBlock + [""] *  (subLevelLines - len(rightTextBlock))

    # compute location of value or link bar for all left and right sub nodes
    #   * left node's value ends at line's width
    #   * right node's value starts after initial spaces
    leftLineWidths    = [ len(line) for line in leftSubLines  ]                            
    rightLineIndents  = [ len(line)-len(line.lstrip(" ")) for line in rightSubLines ]

    # top line value locations, will be used to determine position of current node & link bars
    firstLeftWidth    = (leftLineWidths   + [0])[0]  
    firstRightIndent  = (rightLineIndents + [0])[0] 

    # width of sub node link under node value (i.e. with slashes if any)
    # aims to center link bars under the value if value is wide enough
    # 
    # ValueLine:    v     vv    vvvvvv   vvvvv
    # LinkLine:    / \   /  \    /  \     / \ 
    #
    linkSpacing       = min(stringValueWidth, 2 - stringValueWidth % 2)
    leftLinkBar       = 1 if leftNode  else 0
    rightLinkBar      = 1 if rightNode else 0
    minLinkWidth      = leftLinkBar + linkSpacing + rightLinkBar
    valueOffset       = (stringValueWidth - linkSpacing) // 2

    # find optimal position for right side top node
    #   * must allow room for link bars above and between left and right top nodes
    #   * must not overlap lower level nodes on any given line (allow gap of minSpacing)
    #   * can be offset to the left if lower subNodes of right node 
    #     have no overlap with subNodes of left node                                                                                                                                 
    minSpacing        = 2
    rightNodePosition = fn.reduce(lambda r,i: max(r,i[0] + minSpacing + firstRightIndent - i[1]), \
                                 zip(leftLineWidths,rightLineIndents[0:commonLines]), \
                                 firstLeftWidth + minLinkWidth)

    # extend basic link bars (slashes) with underlines to reach left and right
    # top nodes.  
    #
    #        vvvvv
    #       __/ \__
    #      L       R
    #
    linkExtraWidth    = max(0, rightNodePosition - firstLeftWidth - minLinkWidth )
    rightLinkExtra    = linkExtraWidth // 2
    leftLinkExtra     = linkExtraWidth - rightLinkExtra

    # build value line taking into account left indent and link bar extension (on left side)
    valueIndent       = max(0, firstLeftWidth + leftLinkExtra + leftLinkBar - valueOffset)
    valueLine         = " " * max(0,valueIndent) + stringValue
    slash             = "\\" if inverted else  "/"
    backslash         = "/" if inverted else  "\\"
    uLine             = "¯" if inverted else  "_"

    # build left side of link line
    leftLink          = "" if not leftNode else ( " " * firstLeftWidth + uLine * leftLinkExtra + slash)

    # build right side of link line (includes blank spaces under top node value) 
    rightLinkOffset   = linkSpacing + valueOffset * (1 - leftLinkBar)                      
    rightLink         = "" if not rightNode else ( " " * rightLinkOffset + backslash + uLine * rightLinkExtra )

    # full link line (will be empty if there are no sub nodes)                                                                                                    
    linkLine          = leftLink + rightLink

    # will need to offset left side lines if right side sub nodes extend beyond left margin
    # can happen if left subtree is shorter (in height) than right side subtree                                                
    leftIndentWidth   = max(0,firstRightIndent - rightNodePosition) 
    leftIndent        = " " * leftIndentWidth
    indentedLeftLines = [ (leftIndent if line else "") + line for line in leftSubLines ]

    # compute distance between left and right sublines based on their value position
    # can be negative if leading spaces need to be removed from right side
    mergeOffsets      = [ len(line) for line in indentedLeftLines ]
    mergeOffsets      = [ leftIndentWidth + rightNodePosition - firstRightIndent - w for w in mergeOffsets ]
    mergeOffsets      = [ p if rightSubLines[i] else 0 for i,p in enumerate(mergeOffsets) ]

    # combine left and right lines using computed offsets
    #   * indented left sub lines
    #   * spaces between left and right lines
    #   * right sub line with extra leading blanks removed.
    mergedSubLines    = zip(range(len(mergeOffsets)), mergeOffsets, indentedLeftLines)
    mergedSubLines    = [ (i,p,line + (" " * max(0,p)) )       for i,p,line in mergedSubLines ]
    mergedSubLines    = [ line + rightSubLines[i][max(0,-p):]  for i,p,line in mergedSubLines ]                        

    # Assemble final result combining
    #  * node value string
    #  * link line (if any)
    #  * merged lines from left and right sub trees (if any)
    treeLines = [leftIndent + valueLine] + ( [] if not linkLine else [leftIndent + linkLine] ) + mergedSubLines

    # invert final result if requested
    treeLines = reversed(treeLines) if inverted and isTop else treeLines

    # return intermediate tree lines or print final result
    if isTop : print("\n".join(treeLines))
    else     : return treeLines  